用于处理煜邦电力的已分类按类别分割导出文件。
目录结构：
* workshop/cz_1-9/input   待处理las文件
* workshop/cz_1-9/output  加入类别后的las文件
* workshop/cz_1-9/other   合并或切割的las、csv文件

ToDo   
* 文件列表相关生成，可以直接用文件系统的列表listdir来做；
* linux与windows文件路径间隔符不一致

针对煜邦电力，一段电力通道点云分好类之后，按地物分类拆解成数字文件，文件数量不固定，因为某些区段可能没有某个类别

In [1]:
!pwd

/media/huhongjun/DATA4/workshop_VSubST_LiDAR/deployL-LiDAR-Toolset/ws_PDAL/GitHub-PDAL-exercise


In [6]:
# 此处去掉了地面点【2,ground】，手动写好待处理文件列表与分类列表

datadir='workshop'

classification = [(6,'buildings'), (16,'conductor'), (5,'high_vegetation'), 
         (14,'lake'), (3,'low_vegetation'), (4,'mid_vegetation'), 
         (8,'mkp_ground'), (21,'other'), (23,'other_line'), 
         (12,'road'), (19,'scissors_crossing(down)'), (18,'scissors_crossing(up)'), 
         (20,'shield_wire'), (17,'structures'), (15,'substations')]

def runpipe(pljson):
    pipeline = pdal.Pipeline(pljson)
    pipeline.validate()        # check if our JSON and options were good
    pipeline.loglevel = 8      #really noisy
    count = pipeline.execute() 
    arrays = [] #pipeline.arrays
    metadata = pipeline.metadata
    log = pipeline.log
    
    return (count,arrays,metadata,log)

In [3]:
import os
import pdal

# 1. 按文件名赋值分类
folder = "cz_1-9"

fnlist = []
for i in range(len(classification)) :
    classcode, classname = classification[i]
    
    fnin = os.path.join(datadir,folder,'input',"%s_%s.las"%(folder,classname))
    fnout = os.path.join(datadir,folder,'output',"{}_{:0>2d}.las".format(folder,classcode))
    fnlist.append('"'+fnout+'"')
    
    json =  """
    {  "pipeline": [
      { "type" : "readers.las", "filename" : "%s"      },
      { "type" : "filters.assign",  "assignment" : "Classification[0:32]=%s"      },
      { "type" : "writers.las", "filename" : "%s"      }  ]}
      """ % (fnin,classcode,fnout)
    print(json)
    
    #count,arrays,metdata,log = runpipe(json)
    #print("%2d - Code: %2d, Name: %35s.las, Count: %6d " % (i,classcode,fnout,count))


    {  "pipeline": [
      { "type" : "readers.las", "filename" : "workshop/cz_1-9/input/cz_1-9_buildings.las"      },
      { "type" : "filters.assign",  "assignment" : "Classification[0:32]=6"      },
      { "type" : "writers.las", "filename" : "workshop/cz_1-9/output/cz_1-9_06.las"      }  ]}
      

    {  "pipeline": [
      { "type" : "readers.las", "filename" : "workshop/cz_1-9/input/cz_1-9_conductor.las"      },
      { "type" : "filters.assign",  "assignment" : "Classification[0:32]=16"      },
      { "type" : "writers.las", "filename" : "workshop/cz_1-9/output/cz_1-9_16.las"      }  ]}
      

    {  "pipeline": [
      { "type" : "readers.las", "filename" : "workshop/cz_1-9/input/cz_1-9_high_vegetation.las"      },
      { "type" : "filters.assign",  "assignment" : "Classification[0:32]=5"      },
      { "type" : "writers.las", "filename" : "workshop/cz_1-9/output/cz_1-9_05.las"      }  ]}
      

    {  "pipeline": [
      { "type" : "readers.las", "filename" : "worksho

In [4]:
!pwd

/media/huhongjun/DATA4/workshop_VSubST_LiDAR/deployL-LiDAR-Toolset/ws_PDAL/GitHub-PDAL-exercise


In [6]:
# las文件合并

fnin = ','.join(fnlist)
fnout = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all'))

json_merge = """{
  "pipeline": [%s,
    {
      "type": "filters.merge"
    },
    "%s"
  ]
}
""" % (fnin,fnout)

print(json_merge)
#count,arrays,metdata,log = runpipe(json_merge)
#print("Count: %6d, log: %s " % (count, log))

{
  "pipeline": ["workshop/cz_1-9/output/cz_1-9_06.las","workshop/cz_1-9/output/cz_1-9_16.las","workshop/cz_1-9/output/cz_1-9_05.las","workshop/cz_1-9/output/cz_1-9_14.las","workshop/cz_1-9/output/cz_1-9_03.las","workshop/cz_1-9/output/cz_1-9_04.las","workshop/cz_1-9/output/cz_1-9_08.las","workshop/cz_1-9/output/cz_1-9_21.las","workshop/cz_1-9/output/cz_1-9_23.las","workshop/cz_1-9/output/cz_1-9_12.las","workshop/cz_1-9/output/cz_1-9_19.las","workshop/cz_1-9/output/cz_1-9_18.las","workshop/cz_1-9/output/cz_1-9_20.las","workshop/cz_1-9/output/cz_1-9_17.las","workshop/cz_1-9/output/cz_1-9_15.las",
    {
      "type": "filters.merge"
    },
    "workshop/cz_1-9/other/cz_1-9_merge_all.las"
  ]
}

Count: 22545111, log:  


In [7]:
#import pandas as pd

#df = pd.DataFrame(arrays)
#df.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
# las文件分拆，按长度大小 =》有些拆出来的文件很小，并不是平均比例

fnin = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all'))
fnout = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all_splitter_#'))

json_splitter = """
{
    "pipeline":
        [
            "%s",
            {
                "type":"filters.splitter",
                "length":"2000"
            },
            {
                "type":"writers.las",
                "filename":"%s"
            }
        ]
}
"""% (fnin,fnout)

print(json_splitter)

In [5]:
# las文件分拆，按分拆文件数量,基本上点数均分，但空间上都有

fnin = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all'))
fnout = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all_divider_#'))

json_divide = """
{
    "pipeline":
        [
            "%s",
            {
                "type":"filters.divider",
                "count":"3"
            },
            {
                "type":"writers.las",
                "filename":"%s"
            }
        ]
}
""" % (fnin,fnout)

json = json_divide

print(json)
count,arrays,metdata,log = runpipe(json)
print("Count: %6d, log: %s " % (count, log))


{
    "pipeline":
        [
            "workshop/cz_1-9/other/cz_1-9_merge_all.las",
            {
                "type":"filters.divider",
                "count":"3"
            },
            {
                "type":"writers.las",
                "filename":"workshop/cz_1-9/other/cz_1-9_merge_all_divider_#.las"
            }
        ]
}

Count: 22545111, log:  


In [9]:
# las切片，可以设定文件包含点数量的最大值，得到的切片是空间连续，容量等于或小于设定,也是比较等分

fnin = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all'))
fnout = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all_chipper_#'))

json_chipper = """
{
    "pipeline":
        [
            "%s",
            {
                "type":"filters.chipper",
                "capacity":"8000000"
            },
            {
                "type":"writers.las",
                "filename":"%s"
            }
        ]
}
"""% (fnin,fnout)

json = json_chipper

print(json)
count,arrays,metdata,log = runpipe(json)
print("Count: %6d, log: %s " % (count, log))


{
    "pipeline":
        [
            "workshop/cz_1-9/other/cz_1-9_merge_all.las",
            {
                "type":"filters.chipper",
                "capacity":"8000000"
            },
            {
                "type":"writers.las",
                "filename":"workshop/cz_1-9/other/cz_1-9_merge_all_chipper_#.las"
            }
        ]
}

Count: 22545111, log:  


In [18]:
# las文件格式转换=》las - csv
# "order":"X,Y,Z,Red,Green,Blue,PointSourceId,ScanAngleRank,ScanDirectionFlag,NumberOfReturns,ReturnNumber,Intensity,Classification",

fnin = os.path.join(datadir,folder,'other',"{0}_{1}.las".format(folder,'merge_all_chipper_3'))
fnout = os.path.join(datadir,folder,'other',"{0}_{1}.csv".format(folder,'merge_all_chipper_3'))

json_convert = """
{
  "pipeline":[
    {
      "type":"readers.las",
      "filename":"%s"
    },
    {
      "type":"writers.text",
      "format":"csv",
      "order":"X,Y,Z,Red,Green,Blue",
      "keep_unspecified":"false",
      "filename":"%s"
    }
  ]
}
""" % (fnin,fnout)

json = json_convert

print(json)
count,arrays,metdata,log = runpipe(json)
print("Count: %6d, log: %s " % (count, log))


{
  "pipeline":[
    {
      "type":"readers.las",
      "filename":"workshop/cz_1-9/other/cz_1-9_merge_all_chipper_3.las"
    },
    {
      "type":"writers.text",
      "format":"csv",
      "order":"X,Y,Z,Red,Green,Blue",
      "keep_unspecified":"false",
      "filename":"workshop/cz_1-9/other/cz_1-9_merge_all_chipper_3.csv"
    }
  ]
}

Count: 7515037, log:  


In [ ]:
# 按三维网格下采样，分类和RGB信息会丢失

json_voxelgrid = """
{
    "pipeline":
        [
            "workshop\\\\cz_1-9\\\\output\\\\cz_1-9_merge_chipper_3.las",
            {
                "type":"filters.voxelgrid",
                "leaf_z":"0.4"
            },
            {
                "type":"writers.las",
                "filename":"workshop\\\\cz_1-9\\\\output\\\\cz_1-9_merge_chipper_3_voxelgrid_04.las"
            }
        ]
}
"""

print(json_voxelgrid)

## 选择一个类别的json，执行过滤操作

In [ ]:
import pdal

json = json_merge

pipeline = pdal.Pipeline(json)
pipeline.validate()        # check if our JSON and options were good
pipeline.loglevel = 8      #really noisy
count = pipeline.execute()